In [1]:
import MySQLdb

In [2]:
# world 데이터 베이스
data = ["54.180.179.245", "root", "dss", "world"]
world_db = MySQLdb.connect(*data, charset="utf8")

In [3]:
# 예시
QUERY = """
    SELECT Code, Name, GNP, Population
    FROM country
"""

df = pd.read_sql(QUERY, world_db)
df.tail(2)

,Code,Name,GNP,Population
237,ZMB,Zambia,3377.0,9169000
238,ZWE,Zimbabwe,5951.0,11669000


### Quiz 1
- 20점
- 인구수 1000만이 넘는 국가중에 인구밀도가 가장 높은 상위 3개 국가의 국가코드와 인구밀도를 출력하세요.
- 인구밀도는 소수 셋째자리에서 반올림하여 소수 둘째자리까지 출력하세요.

In [4]:
QUERY1 = """
    SELECT Code, (round(Population / SurfaceArea, 2)) AS pop_per_sur
    FROM country
    WHERE Population > 10000000
    ORDER BY pop_per_sur Desc
    LIMIT 3;
"""

In [5]:
df = pd.read_sql(QUERY1, world_db)
df

,Code,pop_per_sur
0,BGD,896.92
1,TWN,615.01
2,KOR,471.11


In [5]:
df = pd.read_sql(QUERY1, world_db)
df

,code,pop_per_sur
0,BGD,896.92
1,TWN,615.01
2,KOR,471.11


### Quiz 2
- 30점
- 위에서 출력된 결과를 아래와 같이 피봇하여 출력되도록 쿼리를 작성하세요.

In [6]:
QUERY2 = """
    select 
        (select code
         from country
         where population >= 10000000
         order by Population/SurfaceArea desc
         limit 1 ) as rank1,
        (select code
         from country
         where population >= 10000000
         order by Population/SurfaceArea desc
         limit 1, 1 ) as rank2,
        (select code
         from country
         where population >= 10000000
         order by Population/SurfaceArea desc
         limit 2, 1 ) as rank3
    union
    select 
        (select round(Population/SurfaceArea, 2) as pop_per_sur
         from country
         where population >= 10000000
         order by pop_per_sur desc
         limit 1 ) as rank1,
        (select round(Population/SurfaceArea, 2) as pop_per_sur
         from country
         where population >= 10000000
         order by pop_per_sur desc
         limit 1, 1 ) as rank2,
        (select round(Population/SurfaceArea, 2) as pop_per_sur
         from country
         where population >= 10000000
         order by pop_per_sur desc
         limit 2, 1 ) as rank3;
"""

In [7]:
df = pd.read_sql(QUERY2, world_db)
df

,rank1,rank2,rank3
0,BGD,TWN,KOR
1,896.92,615.01,471.11


### Quiz 3
- 50점
- 국가의 인구가 5000만이 넘는 국가중에서 국가별 사용하는언어의 비율이 가장 높은 언어와 사용비율과 사용인구를 출력하세요.
- 사용하는 언어의 비율이 높은 국가의 언어순으로 5개의 언어와 국가를 출력하세요.

In [6]:
QUERY3 = """

SELECT ct.code, la.language, la.percentage, (ct.population * la.percentage) AS language_population
FROM
((SELECT countrycode, language, percentage
FROM countrylanguage
where percentage in (select max(percentage) 
from countrylanguage group by countrycode)) as la
JOIN
(SELECT code, population
from country
where population > 50000000
) as ct
ON ct.code = la.countrycode)
ORDER BY la.percentage DESC
LIMIT 5;

"""

In [7]:
pd.read_sql(QUERY3, world_db)

,code,language,percentage,language_population
0,JPN,Japanese,99.1,1.255736e+10
1,EGY,Arabic,98.8,6.764836e+09
2,BGD,Bengali,97.7,1.261844e+10
3,BRA,Portuguese,97.5,1.658621e+10
4,GBR,English,97.3,5.801357e+09


In [9]:
pd.read_sql(QUERY3, world_db)

,code,language,percentage,language_population
0,JPN,Japanese,99.1,125573572.0
1,EGY,Arabic,98.8,67648362.0
2,BGD,Bengali,97.7,126184431.0
3,BRA,Portuguese,97.5,165862125.0
4,GBR,English,97.3,58013570.0
